In [1]:
import csv
import pandas as pd
import numpy as np
import re
import datetime as dt
import time

In [2]:
df = pd.read_csv("Seattle_Police_Department_911_Incident_Response.csv", parse_dates=[0], infer_datetime_format=True)

In [3]:
# df['Event Clearance Date']

In [94]:
df['fullDate'] = df['Event Clearance Date'].apply(lambda x: str(x).split(' ')[0])
df['ECG'] = df['Event Clearance Group']
# df['eventClearanceSeconds'] = pd.DatetimeIndex(df['Event Clearance Date']).hour *3600 + pd.DatetimeIndex(df['Event Clearance Date']).minute *60
df

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,...,Longitude,Latitude,Incident Location,Initial Type Description,Initial Type Subgroup,Initial Type Group,At Scene Time,fullDate,ECG,eventClearanceSeconds
0,﻿15736,10000246357,2010246357,242.0,FIGHT DISTURBANCE,DISTURBANCES,DISTURBANCES,07/17/2010 08:49:00 PM,3XX BLOCK OF PINE ST,M,...,-122.338147,47.610975,"(47.610975163, -122.338146748)",NaN,NaN,NaN,NaN,07/17/2010,DISTURBANCES,74940.0
1,15737,10000246471,2010246471,65.0,THEFT - MISCELLANEOUS,THEFT,OTHER PROPERTY,07/17/2010 08:50:00 PM,36XX BLOCK OF DISCOVERY PARK BLVD,Q,...,-122.404613,47.658325,"(47.658324899, -122.404612874)",NaN,NaN,NaN,NaN,07/17/2010,OTHER PROPERTY,75000.0
2,15738,10000246255,2010246255,250.0,"MISCHIEF, NUISANCE COMPLAINTS","NUISANCE, MISCHIEF COMPLAINTS","NUISANCE, MISCHIEF",07/17/2010 08:55:00 PM,21XX BLOCK OF 3RD AVE,M,...,-122.342843,47.613551,"(47.613551471, -122.342843234)",NaN,NaN,NaN,NaN,07/17/2010,"NUISANCE, MISCHIEF",75300.0
3,15739,10000246473,2010246473,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,07/17/2010 09:00:00 PM,7XX BLOCK OF ROY ST,D,...,-122.341847,47.625401,"(47.625401388, -122.341846999)",NaN,NaN,NaN,NaN,07/17/2010,TRAFFIC RELATED CALLS,75600.0
4,15740,10000246330,2010246330,250.0,"MISCHIEF, NUISANCE COMPLAINTS","NUISANCE, MISCHIEF COMPLAINTS","NUISANCE, MISCHIEF",07/17/2010 09:00:00 PM,9XX BLOCK OF ALOHA ST,D,...,-122.339709,47.627425,"(47.627424837, -122.339708605)",NaN,NaN,NaN,NaN,07/17/2010,"NUISANCE, MISCHIEF",75600.0
5,15741,10000246477,2010246477,281.0,SUSPICIOUS VEHICLE,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,07/17/2010 09:02:00 PM,30XX BLOCK OF W GOVERNMENT WAY,Q,...,-122.396627,47.661312,"(47.66131158, -122.39662681)",NaN,NaN,NaN,NaN,07/17/2010,SUSPICIOUS CIRCUMSTANCES,75720.0
6,15742,10000246436,2010246436,220.0,MENTAL COMPLAINT,MENTAL CALL,MENTAL HEALTH,07/17/2010 09:04:00 PM,50XX BLOCK OF 57TH AVE S,R,...,-122.263332,47.555956,"(47.555955852, -122.263332426)",NaN,NaN,NaN,NaN,07/17/2010,MENTAL HEALTH,75840.0
7,15743,10000246472,2010246472,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,07/17/2010 09:05:00 PM,56XX BLOCK OF BROOKLYN AVE NE,U,...,-122.314166,47.670345,"(47.670345345, -122.3141656)",NaN,NaN,NaN,NaN,07/17/2010,TRAFFIC RELATED CALLS,75900.0
8,15744,10000246493,2010246493,177.0,LIQUOR VIOLATION - INTOXICATED PERSON,LIQUOR VIOLATIONS,LIQUOR VIOLATIONS,07/17/2010 09:09:00 PM,15XX BLOCK OF 4TH AVE,K,...,-122.336891,47.610440,"(47.610440245, -122.336890515)",NaN,NaN,NaN,NaN,07/17/2010,LIQUOR VIOLATIONS,76140.0
9,15745,10000246478,2010246478,220.0,MENTAL COMPLAINT,MENTAL CALL,MENTAL HEALTH,07/17/2010 09:17:00 PM,100XX BLOCK OF COLLEGE WAY N,N,...,-122.334524,47.702332,"(47.702331629, -122.33452433)",NaN,NaN,NaN,NaN,07/17/2010,MENTAL HEALTH,76620.0


In [5]:
weather = pd.read_csv("WEATHER.csv")

In [6]:
df.shape

(1333837, 21)

In [92]:
weather.head()

,﻿year,month,date,tempHigh,tempAvg,tempLow,dewHigh,dewAvg,dewLow,humidityHigh,...,windHigh,windAvg,windHigh.1,precip,events,m,day,fullDate,clear,notClear
0,2015,Jan,1,43,35,27,28,25,21,81,...,4,0,-,0,,01,01,01/01/2015,0,1
1,2015,Jan,2,44,38,32,39,32,25,86,...,12,4,-,0.03,,01,02,01/02/2015,0,1
2,2015,Jan,3,43,38,33,38,35,30,93,...,7,2,-,0,Rain,01,03,01/03/2015,1,0
3,2015,Jan,4,54,48,41,48,41,37,89,...,21,7,32,0.22,Rain,01,04,01/04/2015,1,0
4,2015,Jan,5,57,56,54,52,50,48,88,...,20,14,31,0.07,Rain,01,05,01/05/2015,1,0


In [8]:
import calendar
weather.shape
list(weather.columns.values)

['\ufeffyear',
 'month',
 'date',
 'tempHigh',
 'tempAvg',
 'tempLow',
 'dewHigh',
 'dewAvg',
 'dewLow',
 'humidityHigh',
 'humidityAvg',
 'humidityLow',
 'seaLvlHigh',
 'seaLvlAvg',
 'seaLvlLow',
 'visibilityHigh',
 'visibilityAvg',
 'visibilityLow',
 'windHigh',
 'windAvg',
 'windHigh.1',
 'precip',
 'events']

In [38]:
months_map = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr':'04', 'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct':'10', 'Nov': '11', 'Dec': '12'}
# months_map = {'Jan': '1', 'Feb': '2', 'Mar': '3', 'Apr':'4', 'May': '5', 'Jun': '6', 'Jul': '7', 'Aug': '8', 'Sep': '9', 'Oct':'10', 'Nov': '11', 'Dec': '12'}

In [39]:
weather['m'] = weather['month'].apply(lambda x: months_map[x])

In [40]:
weather[200:]

,﻿year,month,date,tempHigh,tempAvg,tempLow,dewHigh,dewAvg,dewLow,humidityHigh,...,windHigh,windAvg,windHigh.1,precip,events,m,day,fullDate,clear,notClear
200,2015,Jul,20,82,74,66,58,54,53,75,...,13,8,21,0,,07,20,7/20/2015,0,1
201,2015,Jul,21,75,68,62,56,52,49,78,...,9,3,-,0.13,Rain,07,21,7/21/2015,1,0
202,2015,Jul,22,77,68,61,52,50,48,70,...,10,4,-,0,,07,22,7/22/2015,0,1
203,2015,Jul,23,81,71,62,51,50,48,65,...,7,2,-,0,,07,23,7/23/2015,0,1
204,2015,Jul,24,77,67,59,59,53,50,76,...,13,6,24,0.01,Rain,07,24,7/24/2015,1,0
205,2015,Jul,25,70,66,62,61,54,49,88,...,12,4,16,0.02,Rain,07,25,7/25/2015,1,0
206,2015,Jul,26,73,66,60,56,52,49,80,...,8,2,-,0.1,Rain,07,26,7/26/2015,1,0
207,2015,Jul,27,77,65,55,54,51,45,84,...,7,2,-,0.01,,07,27,7/27/2015,0,1
208,2015,Jul,28,82,72,61,53,51,47,72,...,9,3,-,0,,07,28,7/28/2015,0,1
209,2015,Jul,29,89,75,62,55,51,48,75,...,10,3,18,0,,07,29,7/29/2015,0,1


In [54]:
weather['day'] = weather['date'].apply(lambda x: ('0' + str(x)) if x < 10 else str(x) )
# weather['day'] = weather['date'].apply(lambda x: x)
weather['fullDate'] = weather.apply(lambda x: str(x['m']) + '/' + str(x['day']) + '/' + str(x['\ufeffyear']), axis=1 )
#If the apply doesn't work you may have to pass 'axis=1' after the lambda
#(pass in df['weather'] or whatever your column is called)
weather['clear'] = weather['events'].apply(lambda x: 1 if ('Rain' in str(x) or 'Snow' in str(x) or 'Fog' in str(x) or 'Thunderstorm' in str(x)) else 0)
weather['notClear'] = weather['clear'].apply(lambda x: 1 if x is 0 else 0)

In [13]:
weather[2200:]

,﻿year,month,date,tempHigh,tempAvg,tempLow,dewHigh,dewAvg,dewLow,humidityHigh,...,windHigh,windAvg,windHigh.1,precip,events,m,day,fullDate,clear,notClear
2200,2014,Jan,10,55,51,47,46,42,41,86,...,30,16,41,0.17,Rain,1,10,1/10/2014,1,0
2201,2014,Jan,11,58,52,45,48,42,37,86,...,41,20,56,0.84,Rain,1,11,1/11/2014,1,0
2202,2014,Jan,12,52,47,42,46,41,35,86,...,30,18,39,0.06,Rain,1,12,1/12/2014,1,0
2203,2014,Jan,13,51,51,50,46,45,43,86,...,33,16,45,0,NaN,1,13,1/13/2014,0,1
2204,2014,Jan,14,52,49,45,46,45,42,93,...,13,3,16,T,"Fog , Rain",1,14,1/14/2014,1,0
2205,2014,Jan,15,52,47,42,44,41,39,93,...,13,6,15,0,Fog,1,15,1/15/2014,1,0
2206,2014,Jan,16,44,42,40,41,39,37,96,...,14,6,15,0,Fog,1,16,1/16/2014,1,0
2207,2014,Jan,17,42,40,37,38,37,35,93,...,10,5,13,0,Fog,1,17,1/17/2014,1,0
2208,2014,Jan,18,49,41,33,39,35,30,96,...,14,5,15,0,Fog,1,18,1/18/2014,1,0
2209,2014,Jan,19,43,41,38,40,37,35,93,...,20,6,31,0,Fog,1,19,1/19/2014,1,0


In [14]:
list(df.columns.values)

['CAD CDW ID',
 'CAD Event Number',
 'General Offense Number',
 'Event Clearance Code',
 'Event Clearance Description',
 'Event Clearance SubGroup',
 'Event Clearance Group',
 'Event Clearance Date',
 'Hundred Block Location',
 'District/Sector',
 'Zone/Beat',
 'Census Tract',
 'Longitude',
 'Latitude',
 'Incident Location',
 'Initial Type Description',
 'Initial Type Subgroup',
 'Initial Type Group',
 'At Scene Time',
 'fullDate',
 'ECG']

In [15]:
weather.events.unique()

array([' ', 'Rain', 'Fog', 'Fog , Rain', 'Rain , Thunderstorm', 'Snow',
       'Rain , Snow', 'Fog , Rain , Snow', nan, 'Fog , Snow',
       'Fog , Rain , Thunderstorm', 'Rain , Snow , Thunderstorm',
       'Thunderstorm'], dtype=object)

In [55]:
merged = pd.merge(df, weather, on = 'fullDate', how = 'left')

In [17]:
merged[1000000:] #splines

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,...,visibilityLow,windHigh,windAvg,windHigh.1,precip,events,m,day,clear,notClear
1000000,1953218,14000064651,201464651,177.0,LIQUOR VIOLATION - INTOXICATED PERSON,LIQUOR VIOLATIONS,LIQUOR VIOLATIONS,03/01/2014 03:43:00 PM,17XX BLOCK OF 11 AV,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000001,1953221,14000064572,201464572,63.0,THEFT - CAR PROWL,CAR PROWL,CAR PROWL,03/01/2014 04:01:00 PM,SW EDMUNDS ST / 47 AV SW,W,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000002,1953222,14000064565,201464565,139.0,GANG GRAFFITI,PROPERTY DAMAGE,PROPERTY DAMAGE,03/01/2014 03:46:00 PM,32XX BLOCK OF NE 45 ST,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000003,1953223,14000064557,201464557,220.0,MENTAL COMPLAINT,MENTAL CALL,MENTAL HEALTH,03/01/2014 03:22:00 PM,1 AV / YESLER WY,K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000004,1953224,14000064527,201464527,192.0,MISDEMEANOR WARRANT SERVICE,WARRANT CALLS,ARREST,03/01/2014 03:22:00 PM,28XX BLOCK OF SW BARTON ST,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000005,1953225,14000064503,201464503,201.0,ALARMS - RESIDENTIAL BURGLARY (FALSE),BURGLARY ALARMS (FALSE),FALSE ALARMS,03/01/2014 03:29:00 PM,42XX BLOCK OF E MADISON ST,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000006,1953227,14000060545,201460545,470.0,PARKING VIOLATION (EXCEPT ABANDONED VEHICLES),PARKING VIOLATIONS,TRAFFIC RELATED CALLS,02/25/2014 09:15:00 PM,53XX BLOCK OF 15 AV S,R,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000007,1953229,14000064716,201464716,201.0,ALARMS - RESIDENTIAL BURGLARY (FALSE),BURGLARY ALARMS (FALSE),FALSE ALARMS,03/01/2014 04:16:00 PM,79XX BLOCK OF 13 AV SW,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000008,1953230,14000064713,201464713,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,03/01/2014 04:12:00 PM,4 AV / JEFFERSON ST,K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000009,1953231,14000064709,201464709,450.0,DRIVING WHILE UNDER INFLUENCE (DUI),TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,03/01/2014 04:06:00 PM,15 AV NE / NE 145 ST,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
merged[600000:]

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,...,visibilityLow,windHigh,windAvg,windHigh.1,precip,events,m,day,clear,notClear
600000,1845291,16000354931,2016354931,430.0,MOTOR VEHICLE COLLISION,TRAFFIC RELATED CALLS,MOTOR VEHICLE COLLISION INVESTIGATION,09/30/2016 03:23:53 PM,ELLIOTT AV / BELL ST,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600001,1845292,16000354930,2016354930,470.0,PARKING VIOLATION (EXCEPT ABANDONED VEHICLES),PARKING VIOLATIONS,TRAFFIC RELATED CALLS,09/30/2016 03:16:10 PM,5XX BLOCK OF 1 AV S,K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600002,1784962,16000251010,2016251010,64.0,SHOPLIFT,THEFT,SHOPLIFTING,07/13/2016 03:37:49 PM,6XX BLOCK OF 5 AV S,K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600003,1845293,16000354918,2016354918,430.0,MOTOR VEHICLE COLLISION,TRAFFIC RELATED CALLS,MOTOR VEHICLE COLLISION INVESTIGATION,09/30/2016 03:22:01 PM,1 AV NE / NE 64 ST,J,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600004,1878498,16000413685,2016413685,430.0,MOTOR VEHICLE COLLISION,TRAFFIC RELATED CALLS,MOTOR VEHICLE COLLISION INVESTIGATION,11/15/2016 10:47:45 PM,NW 80 ST / 8 AV NW,J,...,1.0,22.0,11.0,29,1.08,Rain,11,15.0,1.0,0.0
600005,1878339,16000413662,2016413662,350.0,HAZARDS,HAZARDS,HAZARDS,11/15/2016 06:49:18 PM,MAGNOLIA BR / WB,Q,...,1.0,22.0,11.0,29,1.08,Rain,11,15.0,1.0,0.0
600006,1856037,16000373935,2016373935,64.0,SHOPLIFT,THEFT,SHOPLIFTING,10/15/2016 06:26:43 PM,4XX BLOCK OF NE NORTHGATE WY,L,...,2.0,31.0,14.0,39,0.73,"Rain , Thunderstorm",10,15.0,1.0,0.0
600007,1741119,16000175426,2016175426,50.0,"BURGLARY - RESIDENTIAL, UNOCCUPIED",RESIDENTIAL BURGLARIES,BURGLARY,05/19/2016 07:54:36 PM,2XX BLOCK OF NE 139 ST,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600008,1741340,16000175949,2016175949,415.0,BLOCKING VEHICLE,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,05/20/2016 02:57:02 AM,ALASKAN WY VI SB / S ATLANTIC ST,K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600009,1741342,16000175910,2016175910,281.0,SUSPICIOUS VEHICLE,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,05/20/2016 02:48:15 AM,58XX BLOCK OF W GREEN LAKE WY N,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
print(merged['ECG'].unique())
print(len(merged['ECG'].unique()))


['DISTURBANCES' 'OTHER PROPERTY' 'NUISANCE, MISCHIEF '
 'TRAFFIC RELATED CALLS' 'SUSPICIOUS CIRCUMSTANCES' 'MENTAL HEALTH'
 'LIQUOR VIOLATIONS' 'TRESPASS' 'ASSAULTS' 'NARCOTICS COMPLAINTS'
 'ACCIDENT INVESTIGATION' 'SHOPLIFTING' 'PROWLER' 'BURGLARY' 'HAZARDS'
 'PROPERTY DAMAGE' 'CAR PROWL' 'BIKE' 'PROSTITUTION'
 'MISCELLANEOUS MISDEMEANORS' 'ANIMAL COMPLAINTS' 'THREATS, HARASSMENT'
 'PERSON DOWN/INJURY' 'AUTO THEFTS' 'PERSONS - LOST, FOUND, MISSING'
 'FALSE ALARMS' 'ROBBERY' 'ARREST' 'FALSE ALACAD'
 'PROPERTY - MISSING, FOUND' 'RECKLESS BURNING' 'BEHAVIORAL HEALTH'
 'WEAPONS CALLS' 'NUISANCE, MISCHIEF' 'LEWD CONDUCT' 'FRAUD CALLS'
 'HARBOR CALLS' 'MOTOR VEHICLE COLLISION INVESTIGATION'
 'DRIVE BY (NO INJURY)' 'OTHER VICE' 'FAILURE TO REGISTER (SEX OFFENDER)'
 'HOMICIDE' 'VICE CALLS' 'PUBLIC GATHERINGS' nan]
45


In [28]:
list(merged.columns.values)

['CAD CDW ID',
 'CAD Event Number',
 'General Offense Number',
 'Event Clearance Code',
 'Event Clearance Description',
 'Event Clearance SubGroup',
 'Event Clearance Group',
 'Event Clearance Date',
 'Hundred Block Location',
 'District/Sector',
 'Zone/Beat',
 'Census Tract',
 'Longitude',
 'Latitude',
 'Incident Location',
 'Initial Type Description',
 'Initial Type Subgroup',
 'Initial Type Group',
 'At Scene Time',
 'fullDate',
 'ECG',
 '\ufeffyear',
 'month',
 'date',
 'tempHigh',
 'tempAvg',
 'tempLow',
 'dewHigh',
 'dewAvg',
 'dewLow',
 'humidityHigh',
 'humidityAvg',
 'humidityLow',
 'seaLvlHigh',
 'seaLvlAvg',
 'seaLvlLow',
 'visibilityHigh',
 'visibilityAvg',
 'visibilityLow',
 'windHigh',
 'windAvg',
 'windHigh.1',
 'precip',
 'events',
 'm',
 'day',
 'clear',
 'notClear']

In [21]:
merged.max(0)

CAD Event Number          1.600043e+10
General Offense Number    2.012221e+10
Event Clearance Code      6.170000e+02
Census Tract              2.650010e+04
Longitude                -1.222174e+02
Latitude                  4.777798e+01
﻿year                     2.016000e+03
date                      3.100000e+01
tempHigh                  7.200000e+01
tempAvg                   6.600000e+01
tempLow                   6.000000e+01
dewHigh                   6.200000e+01
dewAvg                    5.700000e+01
dewLow                    5.400000e+01
humidityHigh              1.000000e+02
humidityAvg               9.700000e+01
humidityLow               9.300000e+01
seaLvlHigh                3.087000e+01
seaLvlAvg                 3.082000e+01
seaLvlLow                 3.076000e+01
visibilityHigh            1.000000e+01
visibilityAvg             1.000000e+01
visibilityLow             1.000000e+01
windHigh                  3.800000e+01
windAvg                   2.100000e+01
day                      

In [22]:
#weather['day'] = weather['date'].apply(lambda x: ('0' + str(x)) if x < 10 else str(x) )
# df['eventClearanceSeconds'] = df['Event Clearance Date'].apply(lambda x: pd.DatetimeIndex(x).hour * 3600 + pd.DatetimeIndex(x).minute*60)
df['eventClearanceSeconds'] = pd.DatetimeIndex(df['Event Clearance Date']).hour *3600 + pd.DatetimeIndex(df['Event Clearance Date']).minute *60

In [23]:
df

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,...,Longitude,Latitude,Incident Location,Initial Type Description,Initial Type Subgroup,Initial Type Group,At Scene Time,fullDate,ECG,eventClearanceSeconds
0,﻿15736,10000246357,2010246357,242.0,FIGHT DISTURBANCE,DISTURBANCES,DISTURBANCES,07/17/2010 08:49:00 PM,3XX BLOCK OF PINE ST,M,...,-122.338147,47.610975,"(47.610975163, -122.338146748)",NaN,NaN,NaN,NaN,07/17/2010,DISTURBANCES,74940.0
1,15737,10000246471,2010246471,65.0,THEFT - MISCELLANEOUS,THEFT,OTHER PROPERTY,07/17/2010 08:50:00 PM,36XX BLOCK OF DISCOVERY PARK BLVD,Q,...,-122.404613,47.658325,"(47.658324899, -122.404612874)",NaN,NaN,NaN,NaN,07/17/2010,OTHER PROPERTY,75000.0
2,15738,10000246255,2010246255,250.0,"MISCHIEF, NUISANCE COMPLAINTS","NUISANCE, MISCHIEF COMPLAINTS","NUISANCE, MISCHIEF",07/17/2010 08:55:00 PM,21XX BLOCK OF 3RD AVE,M,...,-122.342843,47.613551,"(47.613551471, -122.342843234)",NaN,NaN,NaN,NaN,07/17/2010,"NUISANCE, MISCHIEF",75300.0
3,15739,10000246473,2010246473,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,07/17/2010 09:00:00 PM,7XX BLOCK OF ROY ST,D,...,-122.341847,47.625401,"(47.625401388, -122.341846999)",NaN,NaN,NaN,NaN,07/17/2010,TRAFFIC RELATED CALLS,75600.0
4,15740,10000246330,2010246330,250.0,"MISCHIEF, NUISANCE COMPLAINTS","NUISANCE, MISCHIEF COMPLAINTS","NUISANCE, MISCHIEF",07/17/2010 09:00:00 PM,9XX BLOCK OF ALOHA ST,D,...,-122.339709,47.627425,"(47.627424837, -122.339708605)",NaN,NaN,NaN,NaN,07/17/2010,"NUISANCE, MISCHIEF",75600.0
5,15741,10000246477,2010246477,281.0,SUSPICIOUS VEHICLE,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,07/17/2010 09:02:00 PM,30XX BLOCK OF W GOVERNMENT WAY,Q,...,-122.396627,47.661312,"(47.66131158, -122.39662681)",NaN,NaN,NaN,NaN,07/17/2010,SUSPICIOUS CIRCUMSTANCES,75720.0
6,15742,10000246436,2010246436,220.0,MENTAL COMPLAINT,MENTAL CALL,MENTAL HEALTH,07/17/2010 09:04:00 PM,50XX BLOCK OF 57TH AVE S,R,...,-122.263332,47.555956,"(47.555955852, -122.263332426)",NaN,NaN,NaN,NaN,07/17/2010,MENTAL HEALTH,75840.0
7,15743,10000246472,2010246472,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,07/17/2010 09:05:00 PM,56XX BLOCK OF BROOKLYN AVE NE,U,...,-122.314166,47.670345,"(47.670345345, -122.3141656)",NaN,NaN,NaN,NaN,07/17/2010,TRAFFIC RELATED CALLS,75900.0
8,15744,10000246493,2010246493,177.0,LIQUOR VIOLATION - INTOXICATED PERSON,LIQUOR VIOLATIONS,LIQUOR VIOLATIONS,07/17/2010 09:09:00 PM,15XX BLOCK OF 4TH AVE,K,...,-122.336891,47.610440,"(47.610440245, -122.336890515)",NaN,NaN,NaN,NaN,07/17/2010,LIQUOR VIOLATIONS,76140.0
9,15745,10000246478,2010246478,220.0,MENTAL COMPLAINT,MENTAL CALL,MENTAL HEALTH,07/17/2010 09:17:00 PM,100XX BLOCK OF COLLEGE WAY N,N,...,-122.334524,47.702332,"(47.702331629, -122.33452433)",NaN,NaN,NaN,NaN,07/17/2010,MENTAL HEALTH,76620.0


In [56]:
sample = merged.sample(n=50,random_state = 13)
sample.shape
sample

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,...,visibilityLow,windHigh,windAvg,windHigh.1,precip,events,m,day,clear,notClear
214484,466118,11000338328,2011338328,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,10/15/2011 08:57:00 PM,14XX BLOCK OF NW LEARY WAY,J,...,1.0,15.0,7.0,20,0,NaN,10,15,0.0,1.0
686701,1744935,16000182395,2016182395,176.0,LIQUOR VIOLATION - ADULT,LIQUOR VIOLATIONS,LIQUOR VIOLATIONS,05/24/2016 05:24:35 PM,BROADWAY E / E THOMAS ST,E,...,10.0,9.0,4.0,13,0,NaN,05,24,0.0,1.0
56409,73375,10000330110,2010330110,177.0,LIQUOR VIOLATION - INTOXICATED PERSON,LIQUOR VIOLATIONS,LIQUOR VIOLATIONS,09/20/2010 10:06:00 PM,6XX BLOCK OF QUEEN ANNE AVE N,Q,...,6.0,21.0,12.0,28,0.02,Rain,09,20,1.0,0.0
505841,879789,12000276037,2012276037,470.0,PARKING VIOLATION (EXCEPT ABANDONED VEHICLES),PARKING VIOLATIONS,TRAFFIC RELATED CALLS,08/19/2012 12:36:00 PM,3XX BLOCK OF 27TH AVE S,G,...,6.0,12.0,6.0,15,0,NaN,08,19,0.0,1.0
1285608,1080277,15000141736,2015141736,245.0,"DISTURBANCE, OTHER",DISTURBANCES,DISTURBANCES,04/30/2015 11:37:00 PM,3 AV / PIKE ST,M,...,10.0,9.0,5.0,-,0,,04,30,0.0,1.0
754422,1613635,15000404384,2015404384,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,11/19/2015 02:05:15 PM,11XX BLOCK OF N 130 ST,N,...,10.0,12.0,3.0,21,0.08,Rain,11,19,1.0,0.0
741225,1528225,15000295426,2015295426,101.0,"FORGERY, BAD CHECKS",FRAUD CALLS,FRAUD CALLS,08/24/2015 10:51:00 AM,31XX BLOCK OF W HARLEY ST,Q,...,10.0,10.0,4.0,-,0,,08,24,0.0,1.0
292533,628390,11000195753,2011195753,245.0,"DISTURBANCE, OTHER",DISTURBANCES,DISTURBANCES,06/18/2011 02:59:00 PM,26XX BLOCK OF SW BARTON ST,F,...,2.0,14.0,6.0,17,0.41,Rain,06,18,1.0,0.0
664786,1722424,16000142893,2016142893,171.0,PARKS EXCLUSION,PARKS EXCLUSIONS,MISCELLANEOUS MISDEMEANORS,04/25/2016 12:38:39 PM,16XX BLOCK OF 11 AV,E,...,7.0,13.0,5.0,15,0.11,Rain,04,25,1.0,0.0
1320962,1128795,15000211257,2015211257,NaN,NaN,NaN,NaN,NaN,3XX BLOCK OF 9 AV,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
list(sample.columns.values)

['CAD CDW ID',
 'CAD Event Number',
 'General Offense Number',
 'Event Clearance Code',
 'Event Clearance Description',
 'Event Clearance SubGroup',
 'Event Clearance Group',
 'Event Clearance Date',
 'Hundred Block Location',
 'District/Sector',
 'Zone/Beat',
 'Census Tract',
 'Longitude',
 'Latitude',
 'Incident Location',
 'Initial Type Description',
 'Initial Type Subgroup',
 'Initial Type Group',
 'At Scene Time',
 'fullDate',
 'ECG',
 'eventClearanceSeconds',
 '\ufeffyear',
 'month',
 'date',
 'tempHigh',
 'tempAvg',
 'tempLow',
 'dewHigh',
 'dewAvg',
 'dewLow',
 'humidityHigh',
 'humidityAvg',
 'humidityLow',
 'seaLvlHigh',
 'seaLvlAvg',
 'seaLvlLow',
 'visibilityHigh',
 'visibilityAvg',
 'visibilityLow',
 'windHigh',
 'windAvg',
 'windHigh.1',
 'precip',
 'events',
 'm',
 'day',
 'clear',
 'notClear']

In [91]:
sample['eventClearanceSeconds']

214484     75420.0
686701     62640.0
56409      79560.0
505841     45360.0
1285608    85020.0
754422     50700.0
741225     39060.0
292533     53940.0
664786     45480.0
230890     70740.0
1166156    54000.0
383349     40620.0
1259016    39300.0
72253      85440.0
858691     55260.0
979181     65280.0
1091758    33180.0
971912     79500.0
135596     25320.0
797288     81840.0
1280098    75600.0
1195515     1740.0
979824     65280.0
1265911    12540.0
607914      9000.0
228043     67860.0
518108     17220.0
420249     78600.0
367822     33360.0
1251679    28800.0
219309     39120.0
757241     71700.0
1162611    67560.0
1219724    70080.0
1108717    81480.0
745748      8460.0
903256     81180.0
761114     29880.0
87677      21480.0
371669      2400.0
420087     62460.0
707031     34140.0
679378     10620.0
1114086    64020.0
668921      2040.0
171225     64800.0
539914     14220.0
455820     49980.0
868567     46680.0
Name: eventClearanceSeconds, dtype: float64

In [45]:
sample[49:]

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,...,visibilityLow,windHigh,windAvg,windHigh.1,precip,events,m,day,clear,notClear
868567,1623604,15000421271,2015421271,201.0,ALACAD - RESIDENTIAL BURGLARY (FALSE),BURGLARY ALACAD (FALSE),FALSE ALACAD,12/04/2015 12:58:35 PM,102XX BLOCK OF 42 AV SW,W,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
merged.iloc[785068]

CAD CDW ID                                                   1876734
CAD Event Number                                         16000410639
General Offense Number                                    2016410639
Event Clearance Code                                             430
Event Clearance Description                  MOTOR VEHICLE COLLISION
Event Clearance SubGroup                       TRAFFIC RELATED CALLS
Event Clearance Group          MOTOR VEHICLE COLLISION INVESTIGATION
Event Clearance Date                          11/13/2016 04:57:34 AM
Hundred Block Location                        74XX BLOCK OF 32 AV NW
District/Sector                                                    J
Zone/Beat                                                         J2
Census Tract                                                  3100.4
Longitude                                                   -122.398
Latitude                                                     47.6827
Incident Location                 

In [57]:
from sklearn import linear_model
sample = sample[np.isfinite(df['eventClearanceSeconds'])] 
# sample = sample[np.isfinite(df['day'])]
sample['secondsInDay'] = sample['eventClearanceSeconds'].apply(lambda x: int(x))
sample.shape

C:\Users\Nhungh\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app
C:\Users\Nhungh\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(49, 50)

In [58]:
sample

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,...,windHigh,windAvg,windHigh.1,precip,events,m,day,clear,notClear,secondsInDay
214484,466118,11000338328,2011338328,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,10/15/2011 08:57:00 PM,14XX BLOCK OF NW LEARY WAY,J,...,15.0,7.0,20,0,NaN,10,15,0.0,1.0,75420
686701,1744935,16000182395,2016182395,176.0,LIQUOR VIOLATION - ADULT,LIQUOR VIOLATIONS,LIQUOR VIOLATIONS,05/24/2016 05:24:35 PM,BROADWAY E / E THOMAS ST,E,...,9.0,4.0,13,0,NaN,05,24,0.0,1.0,62640
56409,73375,10000330110,2010330110,177.0,LIQUOR VIOLATION - INTOXICATED PERSON,LIQUOR VIOLATIONS,LIQUOR VIOLATIONS,09/20/2010 10:06:00 PM,6XX BLOCK OF QUEEN ANNE AVE N,Q,...,21.0,12.0,28,0.02,Rain,09,20,1.0,0.0,79560
505841,879789,12000276037,2012276037,470.0,PARKING VIOLATION (EXCEPT ABANDONED VEHICLES),PARKING VIOLATIONS,TRAFFIC RELATED CALLS,08/19/2012 12:36:00 PM,3XX BLOCK OF 27TH AVE S,G,...,12.0,6.0,15,0,NaN,08,19,0.0,1.0,45360
1285608,1080277,15000141736,2015141736,245.0,"DISTURBANCE, OTHER",DISTURBANCES,DISTURBANCES,04/30/2015 11:37:00 PM,3 AV / PIKE ST,M,...,9.0,5.0,-,0,,04,30,0.0,1.0,85020
754422,1613635,15000404384,2015404384,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,11/19/2015 02:05:15 PM,11XX BLOCK OF N 130 ST,N,...,12.0,3.0,21,0.08,Rain,11,19,1.0,0.0,50700
741225,1528225,15000295426,2015295426,101.0,"FORGERY, BAD CHECKS",FRAUD CALLS,FRAUD CALLS,08/24/2015 10:51:00 AM,31XX BLOCK OF W HARLEY ST,Q,...,10.0,4.0,-,0,,08,24,0.0,1.0,39060
292533,628390,11000195753,2011195753,245.0,"DISTURBANCE, OTHER",DISTURBANCES,DISTURBANCES,06/18/2011 02:59:00 PM,26XX BLOCK OF SW BARTON ST,F,...,14.0,6.0,17,0.41,Rain,06,18,1.0,0.0,53940
664786,1722424,16000142893,2016142893,171.0,PARKS EXCLUSION,PARKS EXCLUSIONS,MISCELLANEOUS MISDEMEANORS,04/25/2016 12:38:39 PM,16XX BLOCK OF 11 AV,E,...,13.0,5.0,15,0.11,Rain,04,25,1.0,0.0,45480
230890,488470,11000362924,2011362924,470.0,PARKING VIOLATION (EXCEPT ABANDONED VEHICLES),PARKING VIOLATIONS,TRAFFIC RELATED CALLS,11/07/2011 07:39:00 PM,15XX BLOCK OF TAYLOR AVE N,D,...,17.0,8.0,22,0.01,Rain,11,07,1.0,0.0,70740


In [66]:
# tempHigh                                                          54
# tempAvg                                                           51
# tempLow                                                           47
# dewHigh                                                           49
# dewAvg                                                            45
# dewLow                                                            41
# humidityHigh                                                      89
# humidityAvg                                                       80
# humidityLow                                                       71
# seaLvlHigh                                                      30.2
# seaLvlAvg                                                      30.09
# seaLvlLow                                                      29.99
# visibilityHigh                                                    10
# visibilityAvg                                                     10
# visibilityLow                                                      4
# windHigh                                                          20
# windAvg                                                            8
features = ['secondsInDay', 'clear', 'm', 'day', 'tempAvg', 'dewAvg', 'humidityAvg', 'seaLvlAvg', 'visibilityAvg', 'windAvg']
X = sample[features]
X

,secondsInDay,clear,m,day,tempAvg,dewAvg,humidityAvg,seaLvlAvg,visibilityAvg,windAvg
214484,75420,0.0,10,15,50.0,44.0,80.0,30.09,9.0,7.0
686701,62640,0.0,05,24,61.0,50.0,69.0,30.01,10.0,4.0
56409,79560,1.0,09,20,59.0,53.0,77.0,29.85,10.0,12.0
505841,45360,0.0,08,19,67.0,54.0,70.0,30.04,10.0,6.0
1285608,85020,0.0,04,30,57.0,41.0,57.0,30.25,10.0,5.0
754422,50700,1.0,11,19,44.0,37.0,74.0,30.21,10.0,3.0
741225,39060,0.0,08,24,66.0,48.0,51.0,30.03,10.0,4.0
292533,53940,1.0,06,18,56.0,53.0,89.0,29.94,7.0,6.0
664786,45480,1.0,04,25,52.0,41.0,69.0,30.22,10.0,5.0
230890,70740,1.0,11,07,44.0,39.0,84.0,30.23,10.0,8.0


In [67]:
print(X.shape)
y = sample.ECG#sample['Event Clearance Group']
print(y.shape)
# feature_cols = ['al']
# X = glass[feature_cols]
# y = glass.ri

(49, 10)
(49,)


In [72]:
lr = linear_model.LogisticRegression(solver='newton-cg', multi_class = 'multinomial')
fit = lr.fit(X, y)
preds = lr.predict(X)
lr.get_params()

C:\Users\Nhungh\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'multinomial',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'newton-cg',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [87]:
print(preds)
print(y.head())

['TRAFFIC RELATED CALLS' 'DISTURBANCES' 'SUSPICIOUS CIRCUMSTANCES'
 'TRAFFIC RELATED CALLS' 'DISTURBANCES' 'TRAFFIC RELATED CALLS'
 'FRAUD CALLS' 'DISTURBANCES' 'DISTURBANCES' 'TRAFFIC RELATED CALLS'
 'DISTURBANCES' 'MENTAL HEALTH' 'FRAUD CALLS' 'SUSPICIOUS CIRCUMSTANCES'
 'SUSPICIOUS CIRCUMSTANCES' 'OTHER PROPERTY' 'PROPERTY DAMAGE'
 'TRAFFIC RELATED CALLS' 'DISTURBANCES' 'SUSPICIOUS CIRCUMSTANCES'
 'DISTURBANCES' 'SUSPICIOUS CIRCUMSTANCES' 'TRAFFIC RELATED CALLS'
 'DISTURBANCES' 'BURGLARY' 'SUSPICIOUS CIRCUMSTANCES'
 'SUSPICIOUS CIRCUMSTANCES' 'TRAFFIC RELATED CALLS' 'DISTURBANCES'
 'FALSE ALARMS' 'TRAFFIC RELATED CALLS' 'DISTURBANCES'
 'SUSPICIOUS CIRCUMSTANCES' 'PERSONS - LOST, FOUND, MISSING'
 'SUSPICIOUS CIRCUMSTANCES' 'DISTURBANCES' 'SUSPICIOUS CIRCUMSTANCES'
 'MOTOR VEHICLE COLLISION INVESTIGATION' 'TRAFFIC RELATED CALLS' 'TRESPASS'
 'TRAFFIC RELATED CALLS' 'TRESPASS' 'ASSAULTS' 'DISTURBANCES' 'TRESPASS'
 'SUSPICIOUS CIRCUMSTANCES' 'DISTURBANCES' 'SUSPICIOUS CIRCUMSTANCES'
 'FA

In [69]:
sample['ECG_pred'] = lr.predict(X)

C:\Users\Nhungh\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [85]:
sample.head()

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,...,windAvg,windHigh.1,precip,events,m,day,clear,notClear,secondsInDay,ECG_pred
214484,466118,11000338328,2011338328,460.0,TRAFFIC (MOVING) VIOLATION,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,10/15/2011 08:57:00 PM,14XX BLOCK OF NW LEARY WAY,J,...,7.0,20,0,NaN,10,15,0.0,1.0,75420,TRAFFIC RELATED CALLS
686701,1744935,16000182395,2016182395,176.0,LIQUOR VIOLATION - ADULT,LIQUOR VIOLATIONS,LIQUOR VIOLATIONS,05/24/2016 05:24:35 PM,BROADWAY E / E THOMAS ST,E,...,4.0,13,0,NaN,05,24,0.0,1.0,62640,DISTURBANCES
56409,73375,10000330110,2010330110,177.0,LIQUOR VIOLATION - INTOXICATED PERSON,LIQUOR VIOLATIONS,LIQUOR VIOLATIONS,09/20/2010 10:06:00 PM,6XX BLOCK OF QUEEN ANNE AVE N,Q,...,12.0,28,0.02,Rain,09,20,1.0,0.0,79560,SUSPICIOUS CIRCUMSTANCES
505841,879789,12000276037,2012276037,470.0,PARKING VIOLATION (EXCEPT ABANDONED VEHICLES),PARKING VIOLATIONS,TRAFFIC RELATED CALLS,08/19/2012 12:36:00 PM,3XX BLOCK OF 27TH AVE S,G,...,6.0,15,0,NaN,08,19,0.0,1.0,45360,TRAFFIC RELATED CALLS
1285608,1080277,15000141736,2015141736,245.0,"DISTURBANCE, OTHER",DISTURBANCES,DISTURBANCES,04/30/2015 11:37:00 PM,3 AV / PIKE ST,M,...,5.0,-,0,,04,30,0.0,1.0,85020,DISTURBANCES


In [90]:
feature_cols = ['ECG', 'ECG_pred', 'secondsInDay', 'clear', 'm', 'day', 'tempAvg', 'dewAvg', 'humidityAvg', 'seaLvlAvg', 'visibilityAvg', 'windAvg']
sample[feature_cols].iloc[:10]

,ECG,ECG_pred,secondsInDay,clear,m,day,tempAvg,dewAvg,humidityAvg,seaLvlAvg,visibilityAvg,windAvg
214484,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,75420,0.0,10,15,50.0,44.0,80.0,30.09,9.0,7.0
686701,LIQUOR VIOLATIONS,DISTURBANCES,62640,0.0,05,24,61.0,50.0,69.0,30.01,10.0,4.0
56409,LIQUOR VIOLATIONS,SUSPICIOUS CIRCUMSTANCES,79560,1.0,09,20,59.0,53.0,77.0,29.85,10.0,12.0
505841,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,45360,0.0,08,19,67.0,54.0,70.0,30.04,10.0,6.0
1285608,DISTURBANCES,DISTURBANCES,85020,0.0,04,30,57.0,41.0,57.0,30.25,10.0,5.0
754422,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,50700,1.0,11,19,44.0,37.0,74.0,30.21,10.0,3.0
741225,FRAUD CALLS,FRAUD CALLS,39060,0.0,08,24,66.0,48.0,51.0,30.03,10.0,4.0
292533,DISTURBANCES,DISTURBANCES,53940,1.0,06,18,56.0,53.0,89.0,29.94,7.0,6.0
664786,MISCELLANEOUS MISDEMEANORS,DISTURBANCES,45480,1.0,04,25,52.0,41.0,69.0,30.22,10.0,5.0
230890,TRAFFIC RELATED CALLS,TRAFFIC RELATED CALLS,70740,1.0,11,07,44.0,39.0,84.0,30.23,10.0,8.0


In [ ]:
weather['fullDate']

In [ ]:
df['fullDate'].iloc[30000]

In [71]:
df.groupby('Event Clearance Group').count()

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Date,Hundred Block Location,District/Sector,Zone/Beat,...,Longitude,Latitude,Incident Location,Initial Type Description,Initial Type Subgroup,Initial Type Group,At Scene Time,fullDate,ECG,eventClearanceSeconds
Event Clearance Group,,,,,,,,,,,,,,,,,,,,,
ACCIDENT INVESTIGATION,39269,39269,39269,39269,39269,39269,39267,38793,39260,39269,...,39269,39269,39269,6110,6110,6110,5983,39269,39269,39267
ANIMAL COMPLAINTS,3671,3671,3671,3671,3671,3671,3670,3661,3671,3671,...,3671,3671,3671,2139,2139,2139,1010,3671,3671,3670
ARREST,16422,16422,16422,16422,16422,16422,16412,16341,16387,16422,...,16422,16422,16422,5055,5055,5055,5588,16422,16422,16412
ASSAULTS,23007,23007,23007,23007,23007,23007,22999,22968,22987,23007,...,23007,23007,23007,14003,14003,14003,12074,23007,23007,22999
AUTO THEFTS,29939,29939,29939,29939,29939,29939,29929,29893,29935,29939,...,29939,29939,29939,19213,19213,19213,15324,29939,29939,29929
BEHAVIORAL HEALTH,7603,7603,7603,7603,7603,7603,7601,7603,7602,7603,...,7603,7603,7603,7603,7603,7603,5954,7603,7603,7601
BIKE,4150,4150,4150,4150,4150,4150,4150,4147,4145,4150,...,4150,4150,4150,2858,2858,2858,1908,4150,4150,4150
BURGLARY,38618,38618,38618,38618,38618,38618,38616,38616,38603,38618,...,38618,38618,38618,23720,23720,23720,21511,38618,38618,38616
CAR PROWL,47740,47740,47740,47740,47740,47740,47734,47611,47719,47740,...,47740,47740,47740,30140,30140,30140,21472,47740,47740,47734


In [80]:
print(lr.coef_)
print(len(lr.coef_))

[[ -1.45196018e-04  -3.72969497e-02  -5.60643634e-03  -1.94924228e-01
    2.97220510e-01   1.40398122e-01  -3.17946368e-01  -3.89842444e-02
    9.43217089e-02   9.15643189e-03]
 [ -2.07800838e-04  -3.42011562e-02  -7.33975287e-02  -5.73286704e-01
    3.47019422e-02   1.05124335e-01   8.54876277e-02  -1.96569396e-02
    4.10584736e-02  -1.41927128e-01]
 [  4.39703878e-05   1.09734115e-01   1.98186151e-01   1.35479373e-01
   -2.83584705e-01   1.65131912e-01  -1.26192592e-01   9.32091167e-01
   -1.22960300e+00   1.24130265e-01]
 [  8.24939290e-06   1.13522651e-01   5.96789686e-01  -5.00155163e-01
   -2.82106924e-01   2.07246173e-01   1.40827446e-01  -1.93465463e-01
    1.14699538e-01   3.60739196e-01]
 [  3.16472764e-05   2.08654358e-01   2.80366785e-01   2.22507485e-02
   -3.05300531e-01   1.42062612e-01  -1.69392375e-01   5.88956805e-01
    3.67128909e-01   2.25872587e-01]
 [ -1.64577367e-05   2.83411981e-01   5.54945128e-02   1.79074641e-01
   -8.24565027e-02   8.22187430e-02  -9.15005

In [81]:
sample.groupby('ECG').count()

,CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,...,windAvg,windHigh.1,precip,events,m,day,clear,notClear,secondsInDay,ECG_pred
ECG,,,,,,,,,,,,,,,,,,,,,
ASSAULTS,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,1,1
BURGLARY,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,1,1
DISTURBANCES,10,10,10,10,10,10,10,10,10,10,...,10,10,10,9,10,10,10,10,10,10
FALSE ALACAD,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
FALSE ALARMS,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
FRAUD CALLS,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
LIQUOR VIOLATIONS,5,5,5,5,5,5,5,5,5,5,...,5,5,5,2,5,5,5,5,5,5
MENTAL HEALTH,2,2,2,2,2,2,2,2,2,2,...,2,2,2,1,2,2,2,2,2,2
MISCELLANEOUS MISDEMEANORS,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [83]:
print(lr.intercept_)

[-0.00267026 -0.00192621  0.03915111 -0.00714051  0.03451673  0.02380826
  0.01158095 -0.0103356   0.00834444 -0.02507224  0.00446811 -0.00642686
  0.01625212 -0.02730552  0.00572905 -0.03314785 -0.02982574]
